In [1]:
import nltk
import sklearn
import json
import numpy as np
from collections import defaultdict
from sklearn.cluster import KMeans as skkmeans
import os

In [3]:
path = "C:/Users/MARCU Quentin/Documents/Etudes/M1 SID/Scrapping/Clean_Articles_PMC"
dirs = os.listdir(path)
data = open("Data_PMC.jsonl","w")

for file in dirs:
    try:
        article = open(path + "/" + file, 'r')
        content = article.read()
        data.write(content + "\n")
    except:
        print(file)

clean_PMC_1702.txt
clean_PMC_241.txt
clean_PMC_2567.txt
clean_PMC_2686.txt
clean_PMC_3453.txt
clean_PMC_4100.txt
clean_PMC_4456.txt
clean_PMC_4641.txt
clean_PMC_5446.txt
clean_PMC_5963.txt
clean_PMC_6079.txt
clean_PMC_9203.txt
clean_PMC_9271.txt
clean_PMC_9700.txt


In [5]:
workpath = "C:/Users/MARCU Quentin/Documents/Etudes/M1 SID/Scrapping/"

data=open(workpath + "Data_PMC.jsonl","rb")
docs=[json.loads(s.decode("utf-8")) for s in data.readlines()]

In [6]:
content = [x['content'] for x in docs]

def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if x.isalpha() else " " for x in sent])
    sent = " ".join(sent.split())
    return sent

cleancontent = [sentense2cleanTokens(x) for x in content]

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=2)
tfidf = tfidf_vectorizer.fit_transform(cleancontent)

In [8]:
import random

def distance(a,b):
    return np.linalg.norm(a-b)


def dualkmeans(docs, k, niter=10, nf=2):
    docSize = docs[0].size
    centroids = np.zeros((k,docSize))
    centroid_features = np.array([random.sample([i for i in range(docSize)],k*nf)]).reshape(k,nf)
    for i,cf in enumerate(centroid_features):
        centroids[i,cf] = 1
    for i in range(niter):
        docsInCluster=defaultdict(list)
        # Document assignation
        for idoc,doc in enumerate(docs):
            distances=[distance(np.dot(doc,c),np.zeros(docSize)) for c in centroids]
            docsInCluster[np.argmax(distances)].append(idoc)
        #Cluster redifinition
        seencand=set()
        for ic in docsInCluster:
            candidates=np.argsort(np.ravel(np.average(docs[docsInCluster[ic]],0)))[::-1]
            j=0
            for cand in candidates:
                if (not cand in seencand) and j<nf:
                    centroid_features[ic,j]=cand
                    j+=1
                    seencand.add(cand)
        centroids = np.zeros((k,docSize))
        for ic,cf in enumerate(centroid_features):
            centroids[ic,cf] = 1
    return docsInCluster,centroid_features

In [9]:
clusters,centroid_features = dualkmeans(tfidf.todense(),10, nf=5)

In [10]:
ivocab = tfidf_vectorizer.get_feature_names()
for i, cf in enumerate(centroid_features):
    print("\nCluster label:","-".join([ivocab[x] for x in cf]))


Cluster label: experiments-data-analyzed-available-wrote

Cluster label: soil-plant-microbial-community-effects

Cluster label: climate-change-forest-carbon-changes

Cluster label: study-model-used-models-code

Cluster label: land-use-urban-vegetation-cover

Cluster label: breeding-foraging-nest-nesting-nests

Cluster label: food-environmental-production-management-sustainable

Cluster label: genetic-populations-population-diversity-variation

Cluster label: species-water-the-conservation-we

Cluster label: article-supplementary-material-contains-authorized
